# TransitGraphAI
This notebook uses the contextual transportation graphs preprocessed in the `preprocessing_augmentation.ipynb` notebook. We use reinforcement learning to train an autonomous agent to find the fastest route from one station to the other given a departure time, and without explicit knowledge of the complete timetables.
# Reinforcement learning of route optimizing agents

In [ ]:
import utils.ppo as ppo
import pickle as pkl
from gym.vector import AsyncVectorEnv
import torch

In [ ]:
# Focusing the attempts on the Trentino public transportation graph
data, graph, contextual_graph = pkl.load(open("preprocessed_data/Trentino.pkl", "rb"))

## Time-dependent routing via PPO.

### Description of the environment
We cast transit routing as a Markov Decision Process (MDP) defined over a time-dependent directed graph derived from GTFS data.  
Each state encodes:
- the current station identifier
- the destination identifier
- a compact time-of-day representation using sinusoidal features
- a capped list (top-K) of *feasible*, outgoing options from the current station

Options are constructed on-the-fly: a *trip* edge is feasible only if its departure_time ≥ current_time (earliest-departure constraint), while a *transfer* edge (walking) is always feasible and departs “now” with a duration derived from inter-stop distance and walking speed.  
We normalize option fields to stabilize learning. This design keeps observations compact and informative, reducing partial observability by exposing the agent to a *local timetable frontier* rather than the entire network. We encode the hour cyclically to avoid discontinuities at midnight and normalize all continuous fields to [0,1]. For services spanning >24h (or wrap-around trips), times are stored as minutes and reduced only at the environment boundary, never in the dataset preprocessing step. This prevents mixing timetable semantics with policy normalization and keeps the option generator consistent.

At each step, the agent selects one index corresponding to a possible next move in the options matrix. The environment denormalizes that option to retrieve the next station, the departure time, and the hop duration. The transition first imposes a *wait* for trip edges (0 for transfers) to prevent an agent from choosing already departed trips, then moves the agent to the next station. The episode terminates either on success (destination reached) or on timeout (max_steps). This induces a *time-dependent feasibility surface* that discourages late departures and long waits without hand-crafted rules: feasibility emerges from the option construction itself.

We use a sparse success reward for reaching the destination and light, interpretable shaping: a small per-step penalty to encourage shorter paths, an invalid-action penalty (choosing an out-of-range option or an invalid row), a small positive “progress” shaping when moving to a new station, and a timeout penalty on truncation. This combination provides a well-behaved signal without leaking oracle knowledge (e.g., we do not reveal shortest-time-to-goal). Crucially, transfers are always available but carry duration, making them a flexible fallback that the policy learns to trade off against waiting for the next trip.

In [ ]:
num_envs = 4

graph, station_to_idx, idx_to_station, num_stations = ppo.preprocess_dataframe(
    contextual_graph
)

# Pass the batched spaces to AsyncVectorEnv
envs = AsyncVectorEnv(
    [
        ppo.make_env_fn(graph, station_to_idx, idx_to_station, num_stations, 20, 50)
        for _ in range(num_envs)
    ]
)

### Model description

The policy implements an *option-centric scorer*. We embed station identifiers (shared table for current, destination, and option destinations). For each displayed option row, we concatenate the embedded destination station with continuous fields (normalized departure time, normalized duration, validity bit) and score it via a small MLP, producing per-option logits. This architecture is invariant to the order of options (up to truncation), avoiding a prohibitively large action head. The critic estimates V(s) from the concatenation of the current-station embedding, destination embedding, and time-of-day features, using a lightweight MLP. We intentionally keep the critic ignorant of the option set to encourage general value estimation rather than option-specific scoring.

In [ ]:
embdim = 16
policy = ppo.PolicyNetwork(
    num_stations=num_stations, option_dim=3, embed_dim=embdim, hidden_dim=64
)
critic = ppo.CriticNetwork(num_stations=num_stations, embed_dim=embdim, hidden_dim=64)
optim_policy = torch.optim.Adam(policy.parameters(), lr=3e-4)
optim_critic = torch.optim.Adam(critic.parameters(), lr=1e-3)

## Training loop with reinforcement learning
We train with *Proximal policy optimization* over vectorized environments to decorrelate trajectories. For each episode, we collect a full rollout until all envs terminate or truncate. The policy update maximizes the clipped surrogate with an entropy bonus to maintain exploration, while the critic minimizes MSE between the estimated reward and the actual reward. We run a few policy epochs over the same trajectories (mini-epoching), re-evaluating log-probabilities under the updated policy to form an importance ratio. Gradients are separated for policy/critic with distinct optimizers. We optionally apply advantage normalization within a batch to stabilize the gradient scale.

To overcome cold-start exploration on sparse rewards, we support a curriculum flag (*“easy mode”*) for early episodes: the destination is sampled among the current station’s neighbors at reset. This increases the immediate availability of feasible trip edges, letting the policy first learn *local* time-aware choices (prefer earlier departures with reasonable durations, leverage transfers to break dead ends) before tackling long-range goals. Curriculum is disabled once the policy becomes competent locally. Entropy regularization complements curriculum by discouraging early mode collapse on transfers.


In [ ]:
ppo.ppo_train(
    envs,
    policy,
    critic,
    optim_policy,
    optim_critic,
    num_episodes=3000,
    gamma=0.99,
    clip_eps=0.2,
    critic_coef=0.5,
    entropy_coef=0.01,
    device="cuda",
)

On the Trentino transportation graph, the training loop shows first a decrease in the critic loss, especially during the early *easy mode* phase, and only after some of the policy networks gain the ability to find non-truncated paths with nonnegative rewards. Still, on most epochs, the vast majority of agents do not reach their destination in time, with at best less than 10% that can complete a short route.